# A notebook to copy and clean up PIPS netCDF files for upload to the PERiLS EOL repository

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import warnings
warnings.simplefilter('ignore')
%matplotlib notebook

In [ ]:
# Read in the original PIPS netcdf files

PIPS_input_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/'
PIPS_output_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'
deployment_name = 'IOP3_040522'
PIPS_input_dir = os.path.join(PIPS_input_base_dir, deployment_name, 'netcdf')
PIPS_output_dir = os.path.join(PIPS_output_base_dir, deployment_name, 'netcdf')

# IOP1
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B']
# IOP2
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
# IOP3
PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
parsivel_interval = 10
parsivel_filenames = ['parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, parsivel_interval)
                      for PIPS_name in PIPS_names]
parsivel_filepaths = [os.path.join(PIPS_input_dir, parsivel_filename) for parsivel_filename in parsivel_filenames]
output_parsivel_filepaths = [os.path.join(PIPS_output_dir, parsivel_filename) 
                             for parsivel_filename in parsivel_filenames]
conv_filenames = ['conventional_raw_{}_{}.nc'.format(deployment_name, PIPS_name) for PIPS_name in PIPS_names]
conv_filepaths = [os.path.join(PIPS_input_dir, conv_filename) for conv_filename in conv_filenames]
output_conv_filepaths = [os.path.join(PIPS_output_dir, conv_filename) for conv_filename in conv_filenames]
parsivel_ds_dict = {}
conv_ds_dict = {}
for PIPS_name, parsivel_filepath, conv_filepath in zip(PIPS_names, parsivel_filepaths, conv_filepaths):
    parsivel_ds_dict[PIPS_name] = xr.load_dataset(parsivel_filepath)
    conv_ds_dict[PIPS_name] = xr.load_dataset(conv_filepath)

In [ ]:
PIPS_to_check = 'PIPS1A'

In [ ]:
parsivel_ds = parsivel_ds_dict[PIPS_to_check]
parsivel_ds

In [ ]:
conv_ds = conv_ds_dict[PIPS_to_check]
conv_ds

In [ ]:
# Get variable names to drop

vars_to_drop = []

for var_name in parsivel_ds.data_vars:
    if 'DSD' in var_name:
        vars_to_drop.append(var_name)
    if 'retr' in var_name:
        vars_to_drop.append(var_name)
    if 'at_PIPS' in var_name:
        vars_to_drop.append(var_name)

print(vars_to_drop)

In [ ]:
for dim_name in parsivel_ds.dims:
    print(dim_name)

In [ ]:
# Now go through and remove the unneeded variables

output_parsivel_ds_dict = {}

for PIPS_name in PIPS_names:
    output_parsivel_ds_dict[PIPS_name] = parsivel_ds_dict[PIPS_name].drop_vars(vars_to_drop)    

In [ ]:
# Now go through and remove the unneeded dimensions
dims_to_drop = []
for dim_name in parsivel_ds.dims:
    if 'parameter' in dim_name:
        dims_to_drop.append(dim_name)
    if 'fields' in dim_name:
        dims_to_drop.append(dim_name)

for PIPS_name in PIPS_names:
    output_parsivel_ds_dict[PIPS_name] = output_parsivel_ds_dict[PIPS_name].drop_dims(dims_to_drop) 

In [ ]:
# Now go through and remove unneeded global attributes
attrs_to_drop = []
attrs_to_drop.append('retrieval_wavelength')

for attr_name in parsivel_ds.attrs:
    if 'CG_coeff' in attr_name:
        attrs_to_drop.append(attr_name)
        
for PIPS_name in PIPS_names:
    [output_parsivel_ds_dict[PIPS_name].attrs.pop(attr_name) for attr_name in attrs_to_drop]


In [ ]:
# Now remove the QC attributes from the global list and instead assign them to each of the QC versions of the DSD
# variables. Now, the ones that are in the global list are appropriate for the "rain only" QC (so those variables
# with the "_roqc" tag). The regular "_qc" tagged variables just have strongwindQC, splashingQC, and marginQC set to 1

# First generate the QC attribute dictionaries
ro_qc_dict = {}
qc_dict = {}

attrs_to_drop = []

for attr_name, value in parsivel_ds.attrs.items():
    if 'QC' in attr_name:
        ro_qc_dict[attr_name] = value
        qc_dict[attr_name] = value
        attrs_to_drop.append(attr_name)

# Set rainonlyQC for the regular qc_dict to zero
qc_dict['rainonlyQC'] = 0

# For each PIPS ds, loop through and remove the global QC attributes and apply them to the appropriate variables
# instead
for PIPS_name in PIPS_names:
    [output_parsivel_ds_dict[PIPS_name].attrs.pop(attr_name) for attr_name in attrs_to_drop]
    for var_name in output_parsivel_ds_dict[PIPS_name].data_vars:
        if '_roqc' in var_name:
            for attr_name, value in ro_qc_dict.items():
                output_parsivel_ds_dict[PIPS_name][var_name].attrs[attr_name] = value
        
        if '_qc' in var_name:
            for attr_name, value in qc_dict.items():
                output_parsivel_ds_dict[PIPS_name][var_name].attrs[attr_name] = value

In [ ]:
# Now save to new output directory
for PIPS_name, output_parsivel_filepath in zip(PIPS_names, output_parsivel_filepaths):
    print("Saving {}".format(output_parsivel_filepath))
    output_parsivel_ds_dict[PIPS_name].to_netcdf(output_parsivel_filepath)